In [ ]:
import os

os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.10"
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3.10"
import numpy as np
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
# from pyspark.mllib.feature import HashingTF, IDF

from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix
conf = SparkConf()
conf.set("spark.app.name", "MinHashLSH")
conf.set("spark.debug.maxToStringFields", "100")
conf.set("spark.local.dir", "/dev/shm/pyspark_dir") #TODO: move in arguements
conf.set("spark.driver.memory", "64g")
conf.set("spark.executor.memory", "64g")
spark = SparkSession.builder.config(conf=conf).getOrCreate()


from splearn.rdd import ArrayRDD
from splearn.feature_extraction.text import SparkCountVectorizer, SparkHashingVectorizer
from splearn.feature_extraction.text import SparkTfidfTransformer
from splearn.decomposition import SparkTruncatedSVD
from splearn.pipeline import SparkPipeline


import pandas as pd



num_nodes=1
df = spark.read.option("header", "true").csv("/home/ohadr/database_project_c/test_data/sample.csv")
df = df.withColumn("__id__", F.monotonically_increasing_id()).cache()
records = df.select("__id__", "text").rdd
text_rdd = records.map(lambda x: x[1])
id_rdd = records.map(lambda x: x[0])
# text_rdd.take(10)
# recon_rdd = id_rdd.zip(text_rdd)
# recon_rdd.take(10)

# X = pd.read_csv("/home/ohadr/database_project_c/test_data/sample.csv").text.tolist()

X_rdd = ArrayRDD(text_rdd)  # Get SparkContext from SparkSession

# Use CountVectorizer as requested
spark_vectorizer = SparkCountVectorizer()


dist_pipeline = SparkPipeline((
    ('vect', spark_vectorizer),
    ('tfidf', SparkTfidfTransformer()),
    ('pca', SparkTruncatedSVD(n_components=2))
))

pipeline = dist_pipeline.fit(X_rdd)  # SparseRDD
result_dist = pipeline.transform(X_rdd)  # SparseRDD
recon_rdd = id_rdd.zip(result_dist.unblock())

recon_rdd.zip()

25/03/26 09:53:42 WARN CacheManager: Asked to cache already cached data.


org.apache.spark.api.java.JavaPairRDD@28af7c0b

In [14]:
recon_rdd.collect()

[(0, array([-0.87095063,  0.14698077])),
 (1, array([-0.69496368,  0.18212635])),
 (2, array([-0.21320697, -0.02827101])),
 (3, array([-0.63891718,  0.19311204])),
 (4, array([-0.33683344, -0.60137885])),
 (5, array([-0.1777461 , -0.61820213])),
 (6, array([-0.6378481 ,  0.19451639])),
 (7, array([-0.12278684, -0.32309029])),
 (8, array([-0.52479918, -0.01041242])),
 (9, array([-0.21968482, -0.63012058]))]

In [10]:
import os

os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.10"
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3.10"
import numpy as np
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
# from pyspark.mllib.feature import HashingTF, IDF

from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix
conf = SparkConf()
conf.set("spark.app.name", "MinHashLSH")
conf.set("spark.debug.maxToStringFields", "100")
conf.set("spark.local.dir", "/dev/shm/pyspark_dir") #TODO: move in arguements
conf.set("spark.driver.memory", "64g")
conf.set("spark.executor.memory", "64g")
spark = SparkSession.builder.config(conf=conf).getOrCreate()


In [ ]:
from splearn.rdd import ArrayRDD
from splearn.feature_extraction.text import SparkHashingVectorizer
from splearn.feature_extraction.text import SparkTfidfTransformer
from splearn.pipeline import SparkPipeline

# from sklearn.feature_extraction.text import HashingVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn.pipeline import Pipeline

import pandas as pd
X = pd.read_csv("/home/ohadr/database_project_c/test_data/sample.csv").text.tolist()

X_rdd = ArrayRDD(spark.parallelize(X, 4))  # sc is SparkContext

# local_pipeline = Pipeline((
#     ('vect', HashingVectorizer()),
#     ('tfidf', TfidfTransformer())
# ))
dist_pipeline = SparkPipeline((
    ('vect', SparkHashingVectorizer()),
    ('tfidf', SparkTfidfTransformer())
))

# result_local = local_pipeline.fit_transform(X)
result_dist = dist_pipeline.fit_transform(X_rdd)  # SparseRDD

In [5]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer, IDF, PCA

In [ ]:
# pip install joblibspark

from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from joblibspark import register_spark
from sklearn.utils import parallel_backend

register_spark() # register spark backend

iris = datasets.load_iris()
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svr = svm.SVC(gamma='auto')

clf = GridSearchCV(svr, parameters, cv=5)

with parallel_backend('spark', n_jobs=3):
    clf.fit(iris.data, iris.target)

In [ ]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
# lr = LogisticRegression(maxIter=10, regParam=0.001)
pca  = PCA(k=5, inputCol="features", outputCol="pcaFeatures")
pipeline = Pipeline(stages=[tokenizer, hashingTF, pca])

# Fit the pipeline to training documents.
model = pipeline.fit(df)


In [ ]:


# Prepare test documents, which are unlabeled (id, text) tuples.
# test = spark.createDataFrame([
#     (4, "spark i j k"),
#     (5, "l m n"),
#     (6, "spark hadoop spark"),
#     (7, "apache hadoop")
# ], ["id", "text"])

# Make predictions on test documents and print columns of interest.
prediction = model.transform(test)
selected = prediction.select("id", "text", "probability", "prediction")
for row in selected.collect():
    rid, text, prob, prediction = row
    print(
        "(%d, %s) --> prob=%s, prediction=%f" % (
            rid, text, str(prob), prediction   # type: ignore
        )
    )

In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer

sentenceData = spark.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java could use case classes"),
    (1.0, "Logistic regression models are neat")
], ["label", "sentence"])

tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
wordsData = tokenizer.transform(sentenceData)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)
# alternatively, CountVectorizer can also be used to get term frequency vectors

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

rescaledData.select("label", "features").show()

In [ ]:
from pyspark.ml.feature import CountVectorizer

# Input data: Each row is a bag of words with a ID.
df = spark.createDataFrame([
    (0, "a b c".split(" ")),
    (1, "a b b c a".split(" "))
], ["id", "words"])

# fit a CountVectorizerModel from the corpus.
cv = CountVectorizer(inputCol="words", outputCol="features", vocabSize=3, minDF=2.0)

model = cv.fit(df)

result = model.transform(df)
result.show(truncate=False)

In [ ]:

hashingTF = HashingTF()

tf = hashingTF.transform(text_rdd.map(lambda line: line.split(" ")))

# While applying HashingTF only needs a single pass to the data, applying IDF needs two passes:
# First to compute the IDF vector and second to scale the term frequencies by IDF.
tf.cache()
idf = IDF(minDocFreq=0)
model = idf.fit(tf)
tfidf = model.transform(tf)
mat = RowMatrix(tfidf)
svd = mat.computeSVD(5, computeU=False)
s = svd.s       # The singular values are stored in a local dense vector.
V = svd.V       # The V factor is a local dense matrix.

print("Singular values are: %s" % s)
print("V factor is:\n%s" % V)



In [ ]:
?HashingTF

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

# Prepare training documents from a list of (id, text, label) tuples.
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])

# Configure an ML pipeline, which consists of three stages: tokenizer, hashingTF, and lr.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.001)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

# Fit the pipeline to training documents.
model = pipeline.fit(training)

# Prepare test documents, which are unlabeled (id, text) tuples.
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])

# Make predictions on test documents and print columns of interest.
prediction = model.transform(test)
selected = prediction.select("id", "text", "probability", "prediction")
for row in selected.collect():
    rid, text, prob, prediction = row
    print(
        "(%d, %s) --> prob=%s, prediction=%f" % (
            rid, text, str(prob), prediction   # type: ignore
        )
    )

In [ ]:
U.numCols(),U.numRows()

In [ ]:
U

In [ ]:
V

In [ ]:
V.numRows,V.numCols

In [ ]:
U.

In [ ]:
idf

In [ ]:
tfidf.

In [ ]:

from pyspark import SparkContext
# $example on$
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix
# $example off$

if __name__ == "__main__":
    sc = SparkContext(appName="PythonSVDExample")

    # $example on$
    rows = sc.parallelize([
        Vectors.sparse(5, {1: 1.0, 3: 7.0}),
        Vectors.dense(2.0, 0.0, 3.0, 4.0, 5.0),
        Vectors.dense(4.0, 0.0, 0.0, 6.0, 7.0)
    ])

    mat = RowMatrix(rows)

    # Compute the top 5 singular values and corresponding singular vectors.
    svd = mat.computeSVD(5, computeU=True)
    U = svd.U       # The U factor is a RowMatrix.
    s = svd.s       # The singular values are stored in a local dense vector.
    V = svd.V       # The V factor is a local dense matrix.
    # $example off$
    collected = U.rows.collect()
    print("U factor is:")
    for vector in collected:
        print(vector)
    print("Singular values are: %s" % s)
    print("V factor is:\n%s" % V)
    sc.stop()

In [ ]:
records.take(10)

In [ ]:


from pyspark import SparkContext
# $example on$

# $example off$


sc = SparkContext(appName="TFIDFExample")  # SparkContext

# $example on$
# Load documents (one per line).
documents = sc.textFile("data/mllib/kmeans_data.txt").map(lambda line: line.split(" "))

hashingTF = HashingTF()
tf = hashingTF.transform(documents)

# While applying HashingTF only needs a single pass to the data, applying IDF needs two passes:
# First to compute the IDF vector and second to scale the term frequencies by IDF.
tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf)

# spark.mllib's IDF implementation provides an option for ignoring terms
# which occur in less than a minimum number of documents.
# In such cases, the IDF for these terms is set to 0.
# This feature can be used by passing the minDocFreq value to the IDF constructor.
idfIgnore = IDF(minDocFreq=2).fit(tf)
tfidfIgnore = idfIgnore.transform(tf)
# $example off$

print("tfidf:")
for each in tfidf.collect():
    print(each)

print("tfidfIgnore:")
for each in tfidfIgnore.collect():
    print(each)

sc.stop()